# Element Parameter Detection

## Setup

In [ ]:
%load_ext autoreload

import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch
from numpy.random import default_rng
from tqdm.notebook import tqdm
from ipywidgets import interact 
import gc

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path

import ifcopenshell
import open3d as o3d

from src.elements import *
from src.ifc import *
from src.preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance, get_oriented_bbox_from_points
from src.icp import icp_finetuning
from src.chamfer import *
from src.utils import *
from src.plots import plot_error_graph, plot_parameter_errors
from src.pca import testset_PCA
from src.finetune import chamfer_fine_tune, mahalanobis_fine_tune
from src.cloud import add_noise


random.seed = 42
rng = default_rng()

### sphere morphing

In [ ]:
# visualise a list of point clouds as an animation using open3d
# use ctrl+c to copy and ctrl+v to set camera and zoom inside visualiser
def create_point_cloud_animation(cloud_list, loss_func, save_image=False, colours=None):
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    cloud = cloud_list[0]
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(cloud)
    if colours is not None:
        point_cloud.colors = o3d.utility.Vector3dVector(colours[0])
    vis.add_geometry(point_cloud)
    stops = [9,39,99,299,999]

    for i in range(len(cloud_list)):
        time.sleep(0.01 + 0.05/(i/10+1))
        cloud = cloud_list[i]
        point_cloud.points = o3d.utility.Vector3dVector(cloud)
        if colours is not None:
            point_cloud.colors = o3d.utility.Vector3dVector(colours[i])
        vis.update_geometry(point_cloud)
        vis.poll_events()
        vis.update_renderer()
        if save_image and i in stops:
            vis.capture_screen_image("sphere/" + loss_func + str(i) + ".jpg", do_render=True)
    vis.destroy_window()

    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Info)


In [ ]:
def calc_direct(x, y):
    return torch.sum(torch.square(x-y))

In [ ]:
%autoreload 2

# morph a sphere into the shape of an input point cloud
# by optimising chamfer loss iteratively
# total points = num_points**2
def morph_sphere(src_pcd_tensor, num_points, iterations, learning_rate, stops=[],
                 loss_func= "chamfer", measure_consistency=True, sphere=True, return_assignment=True):
    
    cuda = torch.device("cuda")
    if sphere:
        # gnerate sphere
        # Generate spherical coordinates
        theta = np.linspace(0, 2 * np.pi, num_points)
        phi = np.linspace(0, np.pi, num_points)

        # Create a meshgrid from spherical coordinates
        theta, phi = np.meshgrid(theta, phi)

        # Convert spherical coordinates to Cartesian coordinates
        x = np.sin(phi) * np.cos(theta)
        y = np.sin(phi) * np.sin(theta)
        z = np.cos(phi)

        # Stack the coordinates to form a 3D point cloud and reshape to (num_points * num_points, 3)
        sphere_points = np.stack([x, y, z], axis=-1).reshape(-1, 3)
        sphere_points = np.array([sphere_points for i in range(len(src_pcd_tensor))])
        sphere_points = torch.tensor(sphere_points, device=cuda, 
                                     requires_grad=True)
    else:
        sphere_points = torch.rand(1, num_points**2, 3, device=cuda, 
                                   dtype=torch.double, requires_grad=True)
    
    # optimise
    optimizer = torch.optim.Adam([sphere_points], lr=learning_rate)
    intermediate, losses, assingments = [], [], []
    chamferDist = ChamferDistance()
    assignments = []

    for i in tqdm(range(iterations)):
        optimizer.zero_grad()
        
        if loss_func == "chamfer":
            nn = chamferDist(
                src_pcd_tensor, sphere_points, bidirectional=True, return_nn=True)
            loss = torch.sum(nn[1].dists) + torch.sum(nn[0].dists)
            assignment = [nn[0].idx[:,:,0].detach().cpu().numpy(), nn[1].idx[:,:,0].detach().cpu().numpy()]
        elif loss_func == "emd":
            loss, assignment = calc_emd(sphere_points, src_pcd_tensor, 0.05, 50)
            assignment = assignment.detach().cpu().numpy()
        elif loss_func == "direct":
            loss = calc_direct(sphere_points, src_pcd_tensor)
            assignment = None
        elif loss_func == "pair":
            loss, assignment = get_pair_loss_clouds_tensor(src_pcd_tensor, sphere_points, add_pair_loss=True, it=i)
        elif loss_func == "jittery":
            loss = get_jittery_cd_tensor(src_pcd_tensor, sphere_points, k=1, it=i)
        elif loss_func == "self":
            loss = get_self_cd_tensor(src_pcd_tensor, sphere_points)
        elif loss_func == "reverse":
            loss, assignment = calc_reverse_weighted_cd_tensor(src_pcd_tensor, sphere_points, return_assignment=True, k=32)
        elif loss_func == "prob":
            loss, assignment = calc_pairing_probabilty_loss_tensor(src_pcd_tensor, sphere_points, k=64)
        elif loss_func == "balanced":
            loss, assignment = calc_balanced_chamfer_loss_tensor(src_pcd_tensor, sphere_points, return_assignment=True, k=32)
        elif loss_func == "single":
            loss, assignment = calc_balanced_single_chamfer_loss_tensor(src_pcd_tensor, sphere_points, return_assignment=True, k=32)
        elif loss_func == "infocd":
            loss, assignment = calc_cd_like_InfoV2(src_pcd_tensor, sphere_points, return_assignment=True)
        elif loss_func == "density":
            loss = calc_relative_density_loss_tensor(src_pcd_tensor, sphere_points, return_assignment=False)
        else:
            print("unspecified loss")
            
        #print("a", assignment[0].shape)
        loss.backward()
        optimizer.step()
        #print("iteration", i, "loss", loss.item())
        
        if i in stops:
            intermediate.append(sphere_points.clone())
            losses.append(loss.item())
            if measure_consistency:
                assignments.append(assignment)
            
    # calculate final chamfer loss
    dist = chamferDist(
                src_pcd_tensor, sphere_points, bidirectional=True)
    emd_loss, _ = calc_emd(sphere_points, src_pcd_tensor, 0.05, 50)
    print("final chamfer dist", dist.item(), "emd", emd_loss.item())
    
    # save assignments for analysis
#     if measure_consistency:
#         with open("sphere/assignments_" + loss_func + ".pkl", "wb") as f:
#             pickle.dump(assingments, f)
            
    intermediate = torch.stack(intermediate)
    if return_assignment:
        assignments = assignments
        return intermediate, losses, assignments
    return intermediate, losses

In [ ]:
def run_morph(cld1_name, loss_func):
    cuda = torch.device("cuda")
    cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
    src_pcd_tensor = torch.tensor([cld1], device=cuda)

    iterations = 1000
    #stops = [0, 10, 50, 100, 150, 500, 999]
    stops = [i for i in range(0,iterations,2)]

    #morphed, losses = morph_sphere(src_pcd_tensor, 64, iterations, 0.01, stops, loss_func=loss_func, return_assignment=False)
    morphed, losses = morph_sphere(src_pcd_tensor, 64, iterations, 0.01, stops, measure_consistency=False, loss_func=loss_func, return_assignment=False)
    morphed = torch.flatten(morphed, start_dim=1, end_dim=2)
    morphed = morphed.cpu().detach().numpy()
    #print(morphed.shape)

    # save frames
    with open("sphere/" + loss_func + ".pkl", "wb") as f:
        pickle.dump(morphed, f)

    with open("sphere/loss_" + loss_func + ".pkl", "wb") as f:
        pickle.dump(losses, f)

    # Save the PointCloud to a PCD file
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(morphed[-1])
    o3d.io.write_point_cloud("sphere/sphere_" + loss_func + ".pcd", point_cloud)

In [ ]:
# cld1_name = "sphere/chair.pcd"
# loss_func = "chamfer"
# run_morph(cld1_name, loss_func)

In [ ]:
%autoreload 2

# visualise animation
cld1_name = "sphere/plane1.pcd"
visualise = True
#loss_funcs = [ "chamfer", "emd", "balanced", "reverse", "single"]
#loss_funcs = [ "balanced", "single"]
loss_funcs = [ "density"]
for loss_func in loss_funcs:
    print(loss_func)
    run_morph(cld1_name, loss_func)
    if visualise:
        with open("sphere/" + loss_func + ".pkl", "rb") as f:
            morphed = pickle.load(f)
        colours = visualise_density(morphed, 'plasma_r')
        with open("sphere/" + loss_func + "_dens.pkl", "wb") as f:
            pickle.dump(colours, f)
        #create_point_cloud_animation(cloud_list, loss_func)
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
def view_density(loss_func): 
    with open("sphere/" + loss_func + ".pkl", "rb") as f:
        morphed = pickle.load(f)
    with open("sphere/" + loss_func + "_dens.pkl", "rb") as f:
        colours = pickle.load(f)

    create_point_cloud_animation(morphed, loss_func, True, colours[:,:,:3])
    
interact(view_density, loss_func=[ "density", "balanced", "infocd", "single", "chamfer", "reverse", "emd", "direct"]); 


In [ ]:
torch.cuda.empty_cache()
gc.collect()

# visualise animation
loss_func = "emd"
with open("sphere/" + loss_func + ".pkl", "rb") as f:
    morphed = pickle.load(f)
print(morphed.shape, loss_func)
cloud_list = [m for m in morphed]
#create_point_cloud_animation(cloud_list, loss_func)


In [ ]:
colours = visualise_density(morphed, 'plasma_r')
with open("sphere/" + loss_func + "_dens.pkl", "wb") as f:
    pickle.dump(colours, f)

### Batch optimisation

In [ ]:
# batch sphere optimisation (for metrics)

def sphere_morph_metrics(loss_func, shapenet_path):
    # load shapenet test dataset
    folders = os.listdir(shapenet_path)[7:8]
    print(loss_func)
    cuda = torch.device("cuda")
    chamferDist = ChamferDistance()

    iterations = 1001
    stops = [i for i in range(0,1001,10)]
    chamfer_results = np.zeros(len(stops))
    emd_results = np.zeros(len(stops))
    count = 0
    assignments_folders = []
    
    for fl in folders:
        files = os.listdir(shapenet_path + fl)
        clouds = []
        for cl in files:
            clouds.append(np.array(o3d.io.read_point_cloud(shapenet_path + fl + "/" + cl).points))
        clouds = np.array(clouds)
        clouds = torch.tensor(clouds, device=cuda)
        count += len(clouds)
        #print(clouds.shape)

        # optimise spheres and gather intermediate clouds
        
        morphed, losses, assignments = morph_sphere(clouds, 64, iterations, 0.01, stops, loss_func=loss_func, 
                                       return_assignment=True)
        assignments_folders.append(assignments)

        #calculate chamfer and EMD
        print(morphed.shape)

        # loop through stops
        for i, mr in enumerate(tqdm(morphed)):
            nn = chamferDist(clouds, mr, bidirectional=True, return_nn=True)
            cd_loss = torch.sum(nn[1].dists) + torch.sum(nn[0].dists)
            #cd_assignment = [nn[0].idx[0,:,0].detach().cpu().numpy(), nn[1].idx[0,:,0].detach().cpu().numpy()]
            emd_loss, _ = calc_emd(clouds, mr, 0.05, 50)
            #emd_assignment = emd_assignment.detach().cpu().numpy()

            #print(cd_loss.item(), emd_loss.item())
#             cd_assignments_cat.append(cd_assignment)
#             emd_assignments_cat.append(emd_assignment)
            chamfer_results[i] += cd_loss
            emd_results[i] += emd_loss

        torch.cuda.empty_cache()
        gc.collect()
    
    print(count, chamfer_results[0])
    chamfer_results = chamfer_results/count
    emd_results = emd_results/count

    # save results
    with open("sphere/" + loss_func + "_metrics8.pkl", "wb") as f:
        pickle.dump([chamfer_results, emd_results, assignments_folders], f)


In [ ]:
# downsample
shapenet_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/ShapeNetCompletion/test/complete/"
downsampled_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/ShapeNetCompletion/downsample/"
# folders = os.listdir(shapenet_path)
# choices = np.random.choice(len(points), 4096)

# for fl in tqdm(folders):
#     if not os.path.exists(downsampled_path+fl):
#         os.mkdir(downsampled_path+fl)
        
#     files = os.listdir(shapenet_path + fl)
#     cloud =  o3d.geometry.PointCloud()
#     for cl in files:
#         points = np.array(o3d.io.read_point_cloud(shapenet_path + fl + "/" + cl).points)
#         points = points[choices]
#         cloud.points = o3d.utility.Vector3dVector(points)
#         o3d.io.write_point_cloud(downsampled_path+fl + "/" + cl, cloud)

In [ ]:
loss_func = "balanced"
#loss_func = "emd"

sphere_morph_metrics(loss_func, downsampled_path)

In [ ]:
# create plots
loss_funcs = ["chamfer"]
chamfer_list, emd_list = [], []
for loss_func in loss_funcs:
    with open("sphere/" + loss_func + "_metrics.pkl", "rb") as f:
        chamfer, emd, assignments = pickle.load(f)
        chamfer_list.append(chamfer)
        emd_list.append(emd)
        
plot_dists(chamfer_list, loss_funcs, "chamfer")
plot_dists(emd_list, loss_funcs, "EMD")
        
print(emd_list[-1], chamfer_list[-1])

In [ ]:
# plot losses on same axis
def plot_dists(losses, labels, title):
    x = np.arange(0, len(losses[0]))

#     # scale chamfer distance to be comparable with mahalanobis
#     max = []

#     mahal_dist_max = np.max(mahal_dist_sk)
#     chamfer_dist_max = np.max(chamfer_dist)
#     chamfer_dist = chamfer_dist / chamfer_dist_max * mahal_dist_max

    plt.figure(figsize=(30, 6))
    for i, loss in enumerate(losses):
        plt.plot(x, loss, label=labels[i])

    plt.xlabel("point cloud index")
    plt.ylabel("distance")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
print(assignments[0])

In [ ]:
# load losses
loss_types = ["reverse", "chamfer", "emd", "pair"]
losses = []

for loss_func in loss_types:
    with open("sphere/loss_" + loss_func + ".pkl", "rb") as f:
        losses.append(pickle.load(f))
        
plot_dists(losses, loss_types, "loss function comparison")


#### consistency

In [ ]:
# measure conssistency between forward and backward correspondences for chamfer distance
# optionally compare against the ideal assignment, as measured by EMD
def measure_assignment_consistency(assignment, emd=None):
    reverse_assignment = torch.gather(assignment[0], 0, assignment[1])
    expected = torch.arange(assignment[0].shape[0], device=torch.device("cuda"))
    consistency = torch.sum(torch.eq(expected, reverse_assignment).long())
    print("consistency", consistency.item(), len(torch.unique(assignment[0])), len(torch.unique(assignment[1])))
    
    if emd is not None:
        #print(emd[:5], assignment[0][:5], assignment[1][:5])
        emd_consistency = torch.sum(torch.eq(emd, assignment[0]).long())
        #print("emd_consistency", emd_consistency.item(), len(torch.unique(emd)))


In [ ]:
# compare correspondences
#TODO: include consistency in top5 matches?
loss_func = "emd"
with open("sphere/assignments_" + "emd" + ".pkl", "rb") as f:
    emd_assignment = pickle.load(f)

with open("sphere/assignments_" + loss_func + ".pkl", "rb") as f:
    assignment = pickle.load(f)
    
for i, ass in enumerate(assignment):
    #measure_assignment_consistency(ass, emd_assignment[i][0])
    measure_assignment_consistency(ass)
